In [1]:
import pickle
import pandas as pd
    
def read_bin(filename):
    with open(filename,'rb') as bf:
        bin_data = pd.read_pickle(bf)
    return bin_data

def save_bin(filename,data):
    with open(filename,'wb') as bf:
        pickle.dump(data,bf)

In [2]:
ontology = read_bin("ontology.pickle")

## Wikidata IDからXeno cant IDの取得

In [ ]:
# # id2Xenoの作成
# import requests
# from bs4 import BeautifulSoup
# import csv

# with open('saved_data/id2Xeno.csv', mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file,fieldnames=["wikidata_id", "xeno_canto_id"])#, fieldnames=["wikidata_id", "xeno_canto_id"]
#     writer.writeheader()
    
#     for i in range(0,len(ontology)):
#         print('\r%d / %d' %(i, 50646), end='')
#         try:
#             url = 'https://www.wikidata.org/entity/'+ontology[i]["id"]+'.json'
#             response = requests.get(url)
#             data = response.json()
            
#             the_id = list(data["entities"].keys())[0]#リダイレクト対策
#             claims = data['entities'][the_id]['claims']
#             if 'P2426' in claims:
#                 print('\r%d / %d              ' %(i, 50646), end='')
#                 xeno_canto_id = claims['P2426'][0]['mainsnak']['datavalue']['value']
#                 writer.writerow({"wikidata_id":the_id, "xeno_canto_id":xeno_canto_id})
#             else:
#                 print('\r%d / %d  nodata' %(i, 50646), end='')
#         except:
#             pass


In [ ]:
# protected_data にコピーを作成して守る

## 音声データ固有のID取得

In [ ]:
def soup2page_num(soup):
    page_links = soup.select('nav.results-pages ul li a')
    max_page_number = 1
    
    for link in page_links:
        page_number = link.get_text()
        if page_number.isdigit():  # 数字かどうかを確認
            max_page_number = max(max_page_number, int(page_number))
    return max_page_number

In [ ]:
def url2soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [ ]:
def sound_ids(soup):
    fancybox_a_tags = soup.find_all('a', class_='fancybox', rel='sono')
    sound_ids = [title["title"].split(":")[0] for title in fancybox_a_tags]
    return sound_ids

In [ ]:
import pandas as pd

df = pd.read_csv("saved_data/id2Xeno.csv",header = None)
id2Xeno = dict(zip(df[0], df[1]))

the_ids = list(id2Xeno.keys())

xeno_canto_ids = []
for k in the_ids:
    xeno_canto_ids.append(id2Xeno[k])

In [ ]:
len(the_ids)

In [ ]:
Xeno2sound_ids = dict()

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

for j in range(0,9627):
    species_query = xeno_canto_ids[j].replace(' ', '%20')# 鳥名を検索クエリに
    first_soup = url2soup(f"https://xeno-canto.org/species/"+species_query)
    
    n = soup2page_num(first_soup)
    all_sound_ids = []
    
    for i in range(1,n+1):
        print('\r%d / %d   :  %d / %d        len:%d         ' %(j, 9627, i, n,len(id2sound_ids)), end='')
        soup = url2soup("https://xeno-canto.org/species/"+species_query+"?pg="+str(i))
        all_sound_ids = all_sound_ids + sound_ids(soup)
        
    Xeno2sound_ids[the_ids[j]] = all_sound_ids

In [ ]:
save_bin("saved_data/Xeno2sound_ids.bin",Xeno2sound_ids)

In [ ]:
len(Xeno2sound_ids)

## 音声データダウンロード

In [3]:
Xeno2sound_ids = read_bin("saved_data/Xeno2sound_ids.bin")

In [4]:
ontology = read_bin("ontology.pickle")

In [ ]:
# 順番通りに取得する必要があるためpandasより
# import pandas as pd
# df = pd.read_csv("saved_data/id2Xeno.csv",header = None)
# the_ids = list(df[0])

In [ ]:
import os
os.makedirs('xeno_data', exist_ok=True)
# error = []

In [ ]:
## short_data 対応

In [5]:
import os

def directory_to_dict(path):
    dir_dict = {}
    for root, dirs, files in os.walk(path):
        for directory in dirs:
            sub_dir_path = os.path.join(root, directory)
            dir_dict[directory] = os.listdir(sub_dir_path)
        break
    
    return dir_dict

In [12]:
directory_path = './xeno_data'
result = directory_to_dict(directory_path)

import pandas as pd
df = pd.read_csv("saved_data/id2Xeno.csv",header = None)
the_ids = list(df[0])

nodata_files = []
for the_id in the_ids:
    if the_id in result and the_id in Xeno2sound_ids:
        if len(result[the_id]) != 101:
            if len(result[the_id]) < len(Xeno2sound_ids[the_id]):
                nodata_files.append(the_id)

In [ ]:
# nodata_files = nodata_files[:1000]

In [13]:
len(nodata_files)

5797

In [14]:
len(the_ids)

11052

In [ ]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_sound(sound_id, the_id):
    sound_num = str(sound_id[2:])  # XC削除
    sound_path = os.path.join("xeno_data", the_id, sound_num + ".mp3")
    
    if not os.path.exists(sound_path):
        try:
            audio_response = requests.get(f"https://xeno-canto.org/{sound_num}/download")
            with open(sound_path, 'wb') as f:
                f.write(audio_response.content)
            return f"{sound_num}.mp3 downloaded"
        except Exception as e:
            return f"Error downloading {sound_num}.mp3: {e}"
    else:
        return f"{sound_num}.mp3 already exists"

def download_all_sounds(j,the_id, sound_ids):
    bird_path = os.path.join("xeno_data", the_id)  # 鳥種フォルダ作成
    os.makedirs(bird_path, exist_ok=True)
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(download_sound, sound_id, the_id) for sound_id in sound_ids]
        i = 0
        for future in as_completed(futures):
            print('\r【%s】  %d / %d   :  %d / %d   ...%s                                     ' % (the_id, j, len(nodata_files), i, len(Xeno2sound_ids[the_id]),future.result()), end='')
            i = i+ 1

def cut_over_101(the_id):
    input_list = Xeno2sound_ids[the_id]
    if len(input_list) > 101:
        input_list = input_list[:101]
    return input_list
    
# 使用例
for j in range(0,len(nodata_files)):
    the_id = nodata_files[j]
    if the_id in Xeno2sound_ids:
        ids_list = cut_over_101(the_id)
        download_all_sounds(j,the_id, ids_list)

【Q1272591】  78 / 5797   :  17 / 52   ...802256.mp3 downloaded                                          to.org', port=443): Max retries exceeded with url: /391871/download (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000246F080F7D0>, 'Connection to xeno-canto.org timed out. (connect timeout=None)'))                                     

In [ ]:
l = [1,2,3,4,5]
l[:5]

In [ ]:
# import requests
# import os
# import re

# for j in range(220,len(nodata_files)):
#     the_id = nodata_files[j]
#     bird_path = os.path.join("xeno_data", the_id)# 鳥種フォルダ作成
#     os.makedirs(bird_path, exist_ok=True)

#     i = 0
#     if the_id in Xeno2sound_ids:
#         s = 0
#         for sound_id in Xeno2sound_ids[the_id]:
#             if s > 100:
#                 break
#             else:
#                 try:
#                     print('\r%d / %d   :  %d / %d                  ' % (j, len(Xeno2sound_ids), i, len(Xeno2sound_ids[the_id])), end='')
#                     sound_num = str(sound_id[2:])  # XC削除
#                     sound_path = os.path.join("xeno_data", the_id, sound_num + ".mp3")
                    
#                     # ここでファイルの存在をチェック
#                     if not os.path.exists(sound_path):
#                         audio_response = requests.get("https://xeno-canto.org/" + sound_num + "/download")
#                         with open(sound_path, 'wb') as f:
#                             f.write(audio_response.content)
#                         i = i + 1
#                     else:
#                         print(f"\nFile {sound_num}.mp3 already exists, skipping download.")
                    
#                     s = s + 1
#                 except:
#                     print('\r%d / %d   :  %d / %d  error           ' %(j, len(Xeno2sound_ids), i, len(Xeno2sound_ids[the_id])), end='')
#                     error.append([the_id,sound_id])
#                     s = s + 1

In [ ]:
# error